<a href="https://colab.research.google.com/github/milton-miranda/Classificacao-desbalanceada/blob/main/Case2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case 2: Simulação com diferentes logisticas

In [ ]:
install.package(c("tidyverse", "MASS", "caTools", "caret", "ggplot2", "UBL"))
install.package(c("smotefamily", "boot", "ROCR", "pROC", "gridExtra", "DMwR"))
install.package(c("e1071", "mltools", "tictoc"))

In [ ]:
## Carregar bibliotecas necessárias
library(tidyverse)
library(caret)
library(e1071)
library(randomForest)
library(pROC)
library(ROCR)

## Função para gerar dados desbalanceados
gerar_dados_desbalanceados <- function(n_observacoes = 10000, desbalanceamento = 0.005) {
  n_classe_2 <- round(n_observacoes * desbalanceamento)
  n_classe_1 <- n_observacoes - n_classe_2

  dados <- data.frame(
    Feature1 = rnorm(n_observacoes),
    Feature2 = rnorm(n_observacoes),
    Classe = factor(c(rep(1, n_classe_1), rep(2, n_classe_2)))
  )

  dados <- dados[sample(nrow(dados)), ]  # Embaralhar dados
  return(dados)
}

## Cenários de desbalanceamento
cenarios <- list(
  "0.5%" = 0.005,
  "1%" = 0.01,
  "3.5%" = 0.035
)

## Avaliação de métricas de classificação
evaluar_modelo <- function(modelo, dados_treino, dados_teste) {
  previsoes <- predict(modelo, newdata = dados_teste, type = "response")

  matriz_confusao <- confusionMatrix(as.factor(ifelse(previsoes > 0.5, 2, 1)), dados_teste$Classe, positive = "2")
  auc <- roc(dados_teste$Classe, previsoes)$auc

  list(
    Matriz_Confusao = matriz_confusao,
    AUC = auc,
    Métricas = matriz_confusao$byClass
  )
}

## Loop para rodar o estudo em cada cenário usando Regressão Logística
resultados <- list()
for (cenario in names(cenarios)) {
  cat("\nRodando cenário de desbalanceamento:", cenario, "\n")
  dados <- gerar_dados_desbalanceados(10000, cenarios[[cenario]])

  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  dados_treino <- dados[indices, ]
  dados_teste <- dados[-indices, ]

  cat("\nTreinando modelo: Regressão Logística\n")
  modelo <- glm(Classe ~ ., data = dados_treino, family = binomial)
  resultados[[paste(cenario, "Regressão Logística", sep = " - ")]] <- evaluar_modelo(modelo, dados_treino, dados_teste)
}

## Exibir resultados
for (resultado in names(resultados)) {
  cat("\n---", resultado, "---\n")
  print(resultados[[resultado]]$Matriz_Confusao)
  cat("AUC:", resultados[[resultado]]$AUC, "\n")
  print(resultados[[resultado]]$Métricas)
}

## Plotar Curvas ROC
plot_roc <- function(dados_teste, previsoes, titulo) {
  roc_obj <- roc(dados_teste$Classe, previsoes)
  plot(roc_obj, main = titulo, col = "blue", lwd = 2)
  cat("AUC:", auc(roc_obj), "\n")
}

for (cenario in names(cenarios)) {
  dados <- gerar_dados_desbalanceados(10000, cenarios[[cenario]])
  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  dados_teste <- dados[-indices, ]
  modelo <- glm(Classe ~ ., data = dados[indices, ], family = binomial)
  previsoes <- predict(modelo, newdata = dados_teste, type = "response")
  plot_roc(dados_teste, previsoes, paste("Curva ROC -", cenario))
}

# R Script: Estudo de Simulação com Dados Desbalanceados e Variáveis Categóricas

In [ ]:

## Carregar bibliotecas necessárias
library(tidyverse)
library(caret)
library(glmnet)
library(randomForest)
library(xgboost)
library(pROC)
library(ROCR)
library(tictoc)

## Função para gerar dados desbalanceados categóricos
criar_dados_categoricos <- function(n_observacoes = 1000, proporcao_minoria = 0.005) {
  n_classe_A <- round(n_observacoes * proporcao_minoria)
  n_classe_B <- n_observacoes - n_classe_A

  dados <- data.frame(
    Classe = factor(c(rep("A", n_classe_A), rep("B", n_classe_B))),
    replicate(200, sample(c("X", "Y", "Z"), n_observacoes, replace = TRUE))
  )
  dados <- dados[sample(nrow(dados)), ]  # Embaralhar dados
  return(dados)
}

## Cenários de desbalanceamento
cenarios <- list(
  "0.5%" = 0.005,
  "1%" = 0.01,
  "3.5%" = 0.035
)

## Loop para criar dados, dividir em treino/teste, e aplicar os modelos
resultados <- list()

for (cenario in names(cenarios)) {
  cat("\nRodando cenário de desbalanceamento:", cenario, "\n")
  dados <- criar_dados_categoricos(1000, cenarios[[cenario]])

  # Dividir em treino e teste
  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  treino <- dados[indices, ]
  teste <- dados[-indices, ]

  # Codificação das variáveis categóricas em dummies
  treino_dummies <- dummyVars(Classe ~ ., data = treino)
  treino_codificado <- data.frame(predict(treino_dummies, newdata = treino))
  treino_codificado$Classe <- treino$Classe

  teste_codificado <- data.frame(predict(treino_dummies, newdata = teste))
  teste_codificado$Classe <- teste$Classe

  # Oversampling da classe minoritária
  treino_balanceado <- treino_codificado %>%
    filter(Classe == "A") %>%
    slice_sample(n = sum(treino_codificado$Classe == "B")) %>%
    bind_rows(filter(treino_codificado, Classe == "B"))

  ## Regressão Logística LASSO
  cat("Treinando modelo: Regressão Logística LASSO\n")
  tic()
  x_treino <- model.matrix(Classe ~ . - 1, data = treino_balanceado)
  y_treino <- treino_balanceado$Classe
  lasso_model <- cv.glmnet(x_treino, y_treino, family = "binomial", alpha = 1)
  toc()

  x_teste <- model.matrix(Classe ~ . - 1, data = teste_codificado)
  previsoes_lasso <- predict(lasso_model, newx = x_teste, type = "response")

  ## Random Forest
  cat("Treinando modelo: Random Forest\n")
  tic()
  rf_model <- randomForest(Classe ~ ., data = treino_balanceado)
  toc()
  previsoes_rf <- predict(rf_model, newdata = teste_codificado, type = "prob")[, 2]

  ## XGBoost
  cat("Treinando modelo: XGBoost\n")
  tic()
  xgb_model <- xgboost(data = as.matrix(x_treino), label = as.numeric(y_treino) - 1,
                       nrounds = 100, objective = "binary:logistic", verbose = 0)
  toc()
  previsoes_xgb <- predict(xgb_model, newdata = as.matrix(x_teste))

  ## Avaliação dos modelos
  for (modelo in list("LASSO" = previsoes_lasso, "Random Forest" = previsoes_rf, "XGBoost" = previsoes_xgb)) {
    nome_modelo <- names(modelo)
    previsoes <- modelo[[1]]
    matriz_confusao <- confusionMatrix(as.factor(ifelse(previsoes > 0.5, "A", "B")), teste_codificado$Classe, positive = "A")
    auc <- roc(teste_codificado$Classe, previsoes)$auc

    resultados[[paste(cenario, nome_modelo, sep = " - ")]] <- list(
      Matriz_Confusao = matriz_confusao,
      AUC = auc,
      Métricas = matriz_confusao$byClass
    )
  }
}

## Exibir resultados
for (resultado in names(resultados)) {
  cat("\n---", resultado, "---\n")
  print(resultados[[resultado]]$Matriz_Confusao)
  cat("AUC:", resultados[[resultado]]$AUC, "\n")
  print(resultados[[resultado]]$Métricas)
}

## Plotar Curvas ROC
plot_roc <- function(dados_teste, previsoes, titulo) {
  roc_obj <- roc(dados_teste$Classe, previsoes)
  plot(roc_obj, main = titulo, col = "blue", lwd = 2)
  cat("AUC:", auc(roc_obj), "\n")
}

for (cenario in names(cenarios)) {
  dados <- criar_dados_categoricos(1000, cenarios[[cenario]])
  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  dados_teste <- dados[-indices, ]

  # Regressão Logística LASSO
  modelo_log <- cv.glmnet(model.matrix(Classe ~ . - 1, data = dados[indices, ]),
                          dados[indices, ]$Classe, family = "binomial", alpha = 1)
  previsoes_log <- predict(modelo_log, newx = model.matrix(Classe ~ . - 1, data = dados_teste), type = "response")
  plot_roc(dados_teste, previsoes_log, paste("Curva ROC - Regressão Logística LASSO -", cenario))

  # Random Forest
  modelo_rf <- randomForest(Classe ~ ., data = dados[indices, ])
  previsoes_rf <- predict(modelo_rf, newdata = dados_teste, type = "prob")[, 2]
  plot_roc(dados_teste, previsoes_rf, paste("Curva ROC - Random Forest -", cenario))

  # XGBoost
  modelo_xgb <- xgboost(data = as.matrix(model.matrix(Classe ~ . - 1, data = dados[indices, ])),
                        label = as.numeric(dados[indices, ]$Classe) - 1, nrounds = 100,
                        objective = "binary:logistic", verbose = 0)
  previsoes_xgb <- predict(modelo_xgb, newdata = as.matrix(model.matrix(Classe ~ . - 1, data = dados_teste)))
  plot_roc(dados_teste, previsoes_xgb, paste("Curva ROC - XGBoost -", cenario))
}
